Validación con datos generados sintéticamente
=============================================

No siempre es sencillo tener acceso a repositorios de datos que sean lo suficientemente grandes y con valores realistas (como ser nombre, edad, dirección, fecha de nacimiento, sexo, etc). Disponer de estos conjuntos de datos es útil no solo para aquellas personas que desean probar diferentes técnicas o modelos de aprendizaje automático, sino que también son muy interesantes para validadores de software (testers) que les gustaría tener un conjuntos de datos arbitrariamente grande (aunque con datos ficticios) sobre los cuales ejecutar pruebas de calidad de los modelos de aprendizaje que otros desarrolladores crearon. Esto facilita la posibilidad de probar determinados casos de borde sin necesidad de modificar datos manualmente ni tener que crear de forma artesanal conjuntos de datos para las validaciones.

## Sobre los datos sintéticos

Los conjuntos de datos sintéticos son conjuntos de datos generados a traves de procesos artificiales con el proposito de mantener la privacidad de los datos originales, realizar validaciones de un sistema o pruebas de carga o simplemente verificar un modelo de aprendizaje automático. En general, los datos sintéticos se generan de tal forma que "imitan" la distribución o representación de un conjunto de datos real, pero sin necesidad de revelar ningún detalle del conjunto de datos original. En algunos casos puede ser que los datos generados sintéticamente no tengan el mismo valor que los datos reales, pero existen muchos otros casos donde pueden tener incluso el mismo valor. Muchos modelos de aprendizaje automático son entrenados con datos generados sintéticamente.

Las generación de datos sintéticos se utiliza incluso durante el proceso de entrenamiento de modelos, con técnicas como `SMOTE` que permiten correjir el balance de clases en conjuntos de datos desbalanceados. Sin embargo, en esta sección no nos centraremos en tales típos de técnicas.

## Técnicas para la generación de datos sintéticos

Existen multiples técnicas para la generación de datos sintéticos dependiendo de los requerimientos de negocio y de las características de los conjuntos de datos que estamos buscando. 

### Generarlos desde distribuciones conocidas

Aunque en la práctica no es el caso más habitual, puede ser que conoscamos la distribución de los datos que necesitamos, como ser una distribución normal, exponencial, binomial, etc. En tales casos, muchas librerías de aprendizaje automático nos permiten generar conjuntos de datos de tamaños arbitrales que se condicen con estas distribuciones.

### Utilizando librerías espécificas

Existen librerías específicas para generar conjuntos de datos sintéticos en múltiples lenguajes. En `Python`, librerías como las siguientes pueden ser utilizadas:

 - **faker**: Es una librería que no solo permite la generación de tipos de datos simples, sino que también datos como nombres, direcciones, teléfonos o correos electrónicos.
 - **SymPy:** Es una libería de alta performance que permite generar datos a traves de expresiones simbólicas. 
 - **PyDBGen:** Es una librería que permite generar datos categóricos localizados (localized) para diferentes paises e idiomas. Puede generar conjuntos de datos tanto en archivos como csv o txt como directamente en tablas de bases de datos.
 - **mimesis:** Es una librería similar a faker, aunque provee multiples formas de extender sus funcionalidades.

### Utilizando modelos generativos

Modelos generativos, como son `Variational Autoencoders (VAE)` o `Generative Adversarial Networks (GAN)` puede generar datos sintéticos con un nivel de detallé importante. Muchas técnicas de aumento de datos (data augmentation) descansan en este tipo de modelos.

## Ejemplos

### Mimesis

`Mimesis` es una libería de Python que permite generar conjuntos de datos sintéticos. `Mimesis` no solo permite la generación de tipos de datos simples, sino que también puede generar datos como nombres, direcciones, teléfonos o direcciones de correo electrónico. Esta información la puede generar para diferentes `locations` incluyendo el idioma Español. Lamentablemente, no dispone de una implementación específica para Argentina, siendo las variaciones más aproximadas las de Mexico (es-MX) o España (es-ES). `Mimesis` es extensible, lo que significa que si existe un tipo de dato que no es soportado actualmente, se lo puede implementar para que la librería lo genere. Es ampliamente utilizado por la comunida de código abierto por lo que existen multiples extensiones ya creadas.

Para más información sobre `Mimesis` puede [revisar la documentación oficial](https://mimesis.name/getting_started.html).

```
    !pip install mimesis
```

In [1]:
import mimesis
import mimesis.schema

person = mimesis.Person(mimesis.locales.ES)
numbers = mimesis.Numbers()

Por ejemplo, si quisieramos generar un nombre de una persona generalmente asociado a sexo femenino, podríamos hacer los siguiente:

In [2]:
print(person.full_name(gender=mimesis.enums.Gender.FEMALE))

Nieves Molina


De forma similar, podriamos generar la edad de una persona mayor a 21 años como sigue:

In [3]:
print(numbers.integer_number(start=21, end=100))

42


Esta libreria nos permite generar conjuntos completos de datos de forma rápida indicando un **esquema de datos**. En el siguiente ejemplo, crearemos un conjunto de datos con varios campos, para luego construir un `DataFrame` de `Pandas`:

In [4]:
field = mimesis.schema.Field(mimesis.locales.ES)
description = (
    lambda: {
        'id': field('uuid'),
        'creator': field('full_name', gender=mimesis.enums.Gender.FEMALE),
        'age': field('integer_number', start=21, end=100),
        'timestamp': field('timestamp', posix=False),
        'word': field('text.word'),
        'email': field('person.email', domains=['outlook.com', 'gmail.com', 'yahoo.com'], key=str.lower),
    }
)

sample_schema = mimesis.schema.Schema(schema=description)

Creamos un `Pandas.DataFrame` con 100 muestras:

In [5]:
import pandas as pd

pd.DataFrame(sample_schema.create(iterations=100))

,id,creator,age,timestamp,word,email
0,f911a207-20db-498b-b320-7ca15c929d2a,Verónica Lozano,91,2017-05-04T04:26:33Z,acabar,hemoglobin1812@outlook.com
1,5bb2df8f-d69f-407e-afe5-8f227045d5b4,Mirella Romero,83,2013-07-19T01:02:47Z,encontraste,dunderhead1969@outlook.com
2,14764b8e-8c86-49de-aa79-e07ca8e83c97,Carlota Pardo,70,2008-03-23T10:45:09Z,vieron,against1985@yahoo.com
3,74724ede-9352-4bba-9be2-7bf371d94dd9,Adriana Marcos,32,2002-10-10T17:18:18Z,quienes,bromes1961@gmail.com
4,b2ad7694-5a76-4de9-a1b7-d3abb8036c5d,Rosalía Sanchez,23,2013-03-08T18:57:53Z,perdone,annalisa1948@yahoo.com
...,...,...,...,...,...,...
95,082accf3-2888-4a37-8e78-8f99be8592f1,Caro Calvo,59,2019-02-15T18:02:11Z,películas,enormously1963@outlook.com
96,57de2f47-679e-4198-9d82-73ac9c64f0f8,Desamparados Reyes,22,2018-05-02T19:11:48Z,elección,collodion1986@yahoo.com
97,6c265624-64b4-4d96-b7f9-a91551a1ff73,Isa Ramos,45,2012-10-08T21:24:42Z,tranquila,behead1853@outlook.com
98,b10aad55-b3bf-405b-ab8b-46789a08e43a,Nerea Lopez,45,2007-10-11T11:46:18Z,lados,mischieve1984@outlook.com


### CTGAN

CTGAN es un técnica que ha ganado popularidad ya que se basa en la arquitectura GAN para modelar datos tabulares y muestrear filas condicionalmente a partir de un modelo generado especificamente para crear datos sintéticos. Al momento, esta representa una de las técnicas más avanzadas para la generación de datos sintéticos.

[Xu, Lei and Skoularidou, Maria and Cuesta-Infante, Alfredo and Veeramachaneni, Kalyan. Modeling Tabular data using Conditional GAN. Advances in Neural Information Processing Systems. 2019](https://arxiv.org/abs/1907.00503)

Para demostrar como realizar validaciones utilizando conjunto de datos sintéticos, trabajaremos con el conjunto de datos de 'Pima Indians Diabetes Database' (https://www.openml.org/d/37), proveniente originalmente del National Institute of Diabetes and Digestive and Kidney Diseases. El objetivo del conjunto de datos es el de predecir si un paciente tiene o no diabetes, basado en ciertas mediciones que se incluyen. Este conjunto es interesante para esta demostración porque tiene varias restricciones en cuanto a como se recolectaron los datos. En particular, todos los pacientes son de sexo femenino, de al menos 21 años y de ascendencia india Prima.

In [6]:
import pandas as pd
import sklearn.datasets as datasets

data = datasets.fetch_openml(name='diabetes', version=1)
df = pd.DataFrame(data['data'], columns=data['feature_names'])
df['target'] = data['target']

Veamos como luce el conjunto de datos

In [7]:
df.head(10)

,preg,plas,pres,skin,insu,mass,pedi,age,target
0,6.0,148.0,72.0,35.0,0.0,33.6,0.627,50.0,tested_positive
1,1.0,85.0,66.0,29.0,0.0,26.6,0.351,31.0,tested_negative
2,8.0,183.0,64.0,0.0,0.0,23.3,0.672,32.0,tested_positive
3,1.0,89.0,66.0,23.0,94.0,28.1,0.167,21.0,tested_negative
4,0.0,137.0,40.0,35.0,168.0,43.1,2.288,33.0,tested_positive
5,5.0,116.0,74.0,0.0,0.0,25.6,0.201,30.0,tested_negative
6,3.0,78.0,50.0,32.0,88.0,31.0,0.248,26.0,tested_positive
7,10.0,115.0,0.0,0.0,0.0,35.3,0.134,29.0,tested_negative
8,2.0,197.0,70.0,45.0,543.0,30.5,0.158,53.0,tested_positive
9,8.0,125.0,96.0,0.0,0.0,0.0,0.232,54.0,tested_positive


In [8]:
import warnings

warnings.filterwarnings("ignore")

Entrenemos un modelo basado en la arquitectura de CTGAN para generar sintéticamente instancias pertenecientes a este conjunto de datos. Preste especial atención a los valores especificados en la variable `discrete_columns` donde indicamos aquellas variables que son categoricas. Las restantes variables serán consideradas como continuas.

In [9]:
from ctgan import CTGANSynthesizer

discrete_columns = ['preg', 'age', 'target']

ctgan = CTGANSynthesizer()
ctgan.fit(df, discrete_columns , epochs=5)

In [10]:
ctgan.sample(10)

,preg,plas,pres,skin,insu,mass,pedi,age,target
0,14.0,150.495109,38.928919,32.641231,22.824054,75.231122,0.580614,34.0,tested_positive
1,4.0,174.725586,8.053365,25.389926,-8.499462,46.347632,0.394822,45.0,tested_negative
2,11.0,98.615328,70.139666,8.734297,-1.750806,31.709153,0.766167,62.0,tested_positive
3,1.0,107.990381,76.359883,38.750886,101.828566,45.700789,1.086590,23.0,tested_negative
4,6.0,74.085076,66.234363,-0.216012,-23.033749,-6.380047,-0.036963,66.0,tested_negative
5,11.0,113.149616,123.929728,0.240331,215.650431,42.125721,1.107236,41.0,tested_negative
6,1.0,140.438913,73.185065,4.218364,322.560484,-4.174458,0.455182,24.0,tested_negative
7,11.0,12.590921,69.882593,-0.065983,129.423689,44.400333,1.283018,30.0,tested_negative
8,8.0,165.559505,23.848891,16.881066,-7.283262,63.678178,0.356521,37.0,tested_positive
9,2.0,200.724765,64.242115,1.467709,26.798959,46.813528,0.363256,28.0,tested_negative


.. note:: Note que variables como `age` y `preg` que representan la edad de la persona y la cantidad de veces que ha estado embarazada fueron configuradas como variables categoricas. Esto permite que nuestro modelo 1) no genere valores continuos, como ser 20.5 para la edad, y 2) que no utilice valores que se encuentren fuera de los valores que ya existen en el conjunto de datos. En algunos casos, sobre todo considerando escenarios de testing, este comportamiento podría no ser el deseado.